# Data Mining
## Лабораторная работа 2
### Вариант 2

Работу выполнил: Студент группы М10-414Бки-19 Китили Марк Мвендва 

В данной лабораторной работе мною будет проводиться обработка данных, обучение моделей, подбор гиперпараметров и оценка результатов.

Начнем с обработки данных. Организуем ее проведение через pipeline:

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Загрузка данных из файла
diabetes_data = pd.read_csv('datasets/diabetes.csv')

empty_cols = [column for column in diabetes_data.columns if column not in ['Outcome', 'Pregnancies']]
non_id_cols = [column for column in diabetes_data.columns if column != 'Outcome']


imputer = SimpleImputer(missing_values=0, strategy='median')
empty_transformer = ColumnTransformer(
    [('impute', imputer, empty_cols)],
    remainder='passthrough',    #Остальные столбцы не трогаем
    verbose_feature_names_out=False)    #Не добавляем префикс к названиям столбцов   
empty_transformer.set_output(transform='pandas')  #Выводим в таблицу, а не в массив

scaler = StandardScaler()
scale_transformer = ColumnTransformer(
    [('scale', scaler, non_id_cols)],
    remainder='passthrough',    #Остальные столбцы не трогаем
    verbose_feature_names_out=False)    #Не добавляем префикс к названиям столбцов   
scale_transformer.set_output(transform='pandas')  #Выводим в таблицу, а не в массив

preproc_pipeline = Pipeline([
    ('imputer', empty_transformer),
    ('scaler', scale_transformer)]
)
preproc_pipeline

Pipeline(steps=[('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute',
                                                  SimpleImputer(missing_values=0,
                                                                strategy='median'),
                                                  ['Glucose', 'BloodPressure',
                                                   'SkinThickness', 'Insulin',
                                                   'BMI',
                                                   'DiabetesPedigreeFunction',
                                                   'Age'])],
                                   verbose_feature_names_out=False)),
                ('scaler',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', StandardScaler(),
                                                  ['Pregnancies', 'Glucose',
                                                   'BloodPressure',
                                                   'SkinThickness', 'Insulin',
                                                   'BMI',
                                                   'DiabetesPedigreeFunction',
                                                   'Age'])],
                                   verbose_feature_names_out=False))])

Разделим входные данные на две выборки - обучающую и тестовую:

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit

data = diabetes_data[non_id_cols]
labels = diabetes_data['Outcome']

shuffle = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=12345)

for idx_train, idx_test in shuffle.split(data, labels):
    pass

X_test = data.iloc[idx_test]
y_test = labels.iloc[idx_test]

X_train = data.iloc[idx_train]
y_train = labels.iloc[idx_train]

Разработаем пайплайн для обучения и кросс-валидации

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, roc_auc_score
import pickle

classifiers = {
    'logistic_regression': (LogisticRegression(solver='saga', max_iter = 200, random_state=123),
        {'clf__C': uniform(loc=0.1, scale=10), 'clf__penalty': ['l2', 'l1']}),
    'support_vectors': (SVC(kernel='poly'),
        {'clf__C': uniform(loc=0.1, scale=10), 'clf__degree': randint(2,5), 
        'clf__gamma': uniform(loc = 0, scale = 1), 'clf__coef0': uniform(loc=-1, scale=2)}),
    'knn': (KNeighborsClassifier(),
        {'clf__n_neighbors': randint(2,10), 'clf__p': randint(1,3)}),
    'bayes': (GaussianNB(),
        {'clf__var_smoothing': uniform(loc=1e-10, scale=1)}),
    'random_forest': (RandomForestClassifier(random_state=123),
        {'clf__n_estimators': randint(100,300), 'clf__max_features': ['sqrt', 'log2'], 
        'clf__min_samples_split': randint(2,5)}
    )
}

dataframe = pd.DataFrame(['accuracy', 'recall', 'precision', 'ROC AUC curve'])

for name, values in classifiers.items():
    clf, params = values
    pipeline = Pipeline([
        ("preproc", preproc_pipeline),
        ('clf', clf)]
    )
    model = RandomizedSearchCV(pipeline, params)

    model.fit(X_train, y_train)

    with open(f"saved_models/{name}.pkl", "wb") as file:
        pickle.dump(model, file, protocol=3)
    with open(f"hyperparams/{name}.txt", "w") as file:
        file.write(str(model.best_params_))
    print(name)
    print(model.best_params_, '\n' + '=' * 20)

    y_pred = model.predict(X_test)
    y_true = np.array(y_test)

    dataframe[name] = [accuracy_score(y_true, y_pred), recall_score(y_true, y_pred), 
    precision_score(y_true, y_pred), roc_auc_score(y_true, y_pred)]

    print(  f'Confusion matrix:\n {confusion_matrix(y_true, y_pred)}\n'
    )

dataframe


logistic_regression
{'clf__C': 9.270063244965172, 'clf__penalty': 'l1'} 
Confusion matrix:
 [[87 13]
 [22 32]]

support_vectors
{'clf__C': 1.2889037880068754, 'clf__coef0': 0.9516823119854623, 'clf__degree': 2, 'clf__gamma': 0.2063382250410536} 
Confusion matrix:
 [[87 13]
 [20 34]]

knn
{'clf__n_neighbors': 9, 'clf__p': 1} 
Confusion matrix:
 [[86 14]
 [15 39]]

bayes
{'clf__var_smoothing': 0.5520328604155896} 
Confusion matrix:
 [[88 12]
 [28 26]]

random_forest
{'clf__max_features': 'sqrt', 'clf__min_samples_split': 2, 'clf__n_estimators': 177} 
Confusion matrix:
 [[83 17]
 [15 39]]



,0,logistic_regression,support_vectors,knn,bayes,random_forest
0,accuracy,0.772727,0.785714,0.811688,0.740260,0.792208
1,recall,0.592593,0.629630,0.722222,0.481481,0.722222
2,precision,0.711111,0.723404,0.735849,0.684211,0.696429
3,ROC AUC curve,0.731296,0.749815,0.791111,0.680741,0.776111


В ходе лабораторной работы получены следующие итоги:
 - как и говорилось в первой лабораторной работе, качество датасета оставляет желать лучшего. Максимальная точность составила всего *81.2%* для метода knn. Худшим вариантом оказался наивный Байесовский классификатор - с точностью в *74%*, который в большей степени определяет больных диабетом здоровыми. Для увеличения точности можно убрать строки с пропусками, вместо того чтобы подставлять туда медиану. Но это приведет к еще меньшему датасету, который и так является недостаточным для построения надежной модели.